**RFM Segmentation** is a great method to divide
Customers into equal groups depending on three criteria:

**Recency**:- How much time has elapsed since a customer’s last activity or transaction with the company?

**Frequency**:- How often has a customer transacted with the company during a particular period of time?

**Monetary**:- How much a customer has spent with the brand during a particular period of time?


In [0]:
# Import libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
#import squarify
# Read dataset
df = pd.read_csv('data.csv', encoding = "ISO-8859-1")
# Convert InvoiceDate from object to datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


In [17]:
print('First five rows in the dataset:-')
print(df.head())
print('Last five rows in the dataset:-')
print(df.tail())
print(df.info())
print(df.describe())
print('There are {:,} rows and {:,} columns in the dataset.'
      .format(df.shape[0], df.shape[1]))
print('-----------------------------------------------------')
print('{:,} transactions don\'t have a customer id'
      .format(df[df.CustomerID.isnull()].shape[0]))
print('-----------------------------------------------------')
print('Transactions has been started from {} to {}'.format(df['InvoiceDate'].min(),
                                    df['InvoiceDate'].max()))

First five rows in the dataset:-
  InvoiceNo StockCode  ... CustomerID         Country
0    536365    85123A  ...    17850.0  United Kingdom
1    536365     71053  ...    17850.0  United Kingdom
2    536365    84406B  ...    17850.0  United Kingdom
3    536365    84029G  ...    17850.0  United Kingdom
4    536365    84029E  ...    17850.0  United Kingdom

[5 rows x 8 columns]
Last five rows in the dataset:-
       InvoiceNo StockCode  ... CustomerID         Country
437455    574292     21430  ...    16914.0  United Kingdom
437456    574292     21623  ...    16914.0  United Kingdom
437457    574292     21624  ...    16914.0  United Kingdom
437458    574292     23438  ...    16914.0  United Kingdom
437459    574292     20751  ...        NaN             NaN

[5 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437460 entries, 0 to 437459
Data columns (total 8 columns):
InvoiceNo      437460 non-null object
StockCode      437460 non-null object
Description    436092 non-n